# Converting Words to Anagrams
## Problem Description
Write a function that creates a printout of anagrams of all the words
with a given length in a text with the following specification:
````
anagrams :: Int -> Text -> Dictionary
````

## Solution

In [2]:
:load Pipe

In [3]:
import Pipe
import qualified Data.Char as Char
import qualified Data.List as List
import qualified Data.Set as Set

Defining type synonyms:

In [4]:
type Dictionary = String
type Wort = String
type Text = String
type Label = String

extract the words of length n:

In [5]:
getWords :: Int -> Text -> [Wort]
getWords n text =  [word | word <- words (map Char.toLower text 
    $> filter (\ x -> Char.isLetter x || Char.isSeparator x)), length word == n]

take each word and add a label to it:

In [6]:
addLabel :: Wort -> (Label, Wort)
addLabel word = (List.sort word, word)

sort the list of label-word-tuples in aplphabetical order:

In [7]:
sortLabels :: [(Label, Wort)] -> [(Label, Wort)]
sortLabels = List.sort

replace each group of labelled words with the same label
with a single entry using an accumulator (Set.empty) and a helper function:

In [8]:
groupByLabel :: [(Label, Wort)] -> [(Label, [Wort])]
groupByLabel = groupByHelp Set.empty

groupByHelp :: Set.Set Label -> [(Label, Wort)] -> [(Label, [Wort])]
groupByHelp _ [] = []
groupByHelp set (x:xs) 
  | not (Set.member label set) =
      (label, wort:worte) : groupByHelp (Set.insert label set) xs
  | otherwise = groupByHelp set xs
  where 
      label = fst x
      wort  = snd x
      worte = filter (\ tuple -> fst tuple == label) xs $> map snd $> filter (/= wort)

replace each entry by a string and concatenate the results:

In [9]:
showEntry :: [(Label, [Wort])] -> Dictionary
showEntry [] = []
showEntry (x:xs) = fst x ++ ": " ++ unwords (snd x) ++ "\n" ++ showEntry xs

putting everything together using function application:

In [10]:
anagrams :: Int -> Text -> Dictionary
anagrams n text = getWords n text
                $> map addLabel
                $> sortLabels
                $> groupByLabel
                $> showEntry

In [11]:
anagrams 4 "seid, dies dies!"

"deis: dies seid\n"

In [12]:
printAnagrams :: Int -> FilePath -> FilePath -> IO()
printAnagrams n infile outfile = do
  text <- readFile infile
  writeFile outfile (anagrams n text)
  putStrLn "anagrams done"

In [13]:
printAnagrams 8 "Faust.txt" "Faust.dict"

anagrams done